# When not using any schema or catalog just reading from adls

In [0]:


# List all files at the root of the bucket
files = dbutils.fs.ls("abfss://raw@databdls.dfs.core.windows.net/")
display(files)

In [0]:
# by defaul read all files in folder irrespective of file type
df= spark.read.csv("abfss://raw@databdls.dfs.core.windows.net/")
df_all_csv= spark.read.csv("abfss://raw@databdls.dfs.core.windows.net/*.csv")
df_csv= spark.read.option("header", True).csv("abfss://raw@databdls.dfs.core.windows.net/circuits.csv")
df_csv_with_schema= spark.read.option("header", True).option("inferSchema", True).csv("abfss://raw@databdls.dfs.core.windows.net/circuits.csv")


In [0]:
type(df)

In [0]:
# Show number of input files Spark processed
num_files = df.inputFiles()
print(f"Total files read by Spark: {len(num_files)}")
print("Files read:")
for f in num_files:
    print(f)

In [0]:
num_files = df_all_csv.inputFiles()
print(f"Total files read by Spark: {len(num_files)}")
print("Files read:")
for f in num_files:
    print(f)

In [0]:
num_files = df_csv.inputFiles()
print(f"Total files read by Spark: {len(num_files)}")
print("Files read:")
for f in num_files:
    print(f)

In [0]:
df.show()
df.printSchema()
df.count()

In [0]:
df_csv.show()

In [0]:
df_csv.printSchema()

In [0]:
df_csv.describe().show()

In [0]:
display(df_csv)


In [0]:
# DATa ingestin using Schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col


circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),

                                   StructField("circuitRef", StringType(), True),
                                   StructField("name", StringType(), True),
                                   StructField("location", StringType(), True),
                                   StructField("country", StringType(), True),
                                   StructField("lat", DoubleType(), True),
                                   StructField("lng", DoubleType(), True),
                                   StructField("alt", IntegerType(),True),
                                #    StructField("url", StringType(), True),
                                   ])
df_csv_with_schema = spark.read.option("header", True)  \
.option("treatEmptyValuesAsNulls", "false") \
.schema(circuits_schema).csv("abfss://raw@databdls.dfs.core.windows.net/circuits.csv")

# Check for nulls in 'alt' as spark does not throw error
null_alt_count = df_csv_with_schema.filter(col("alt").isNull()).count()

# if null_alt_count > 0:
#      raise ValueError(f"'alt' column has {null_alt_count} null values but schema says nullable=False.")

# For IntegerType, DoubleType, stringtype, etc. spark turns empty value as null

df_csv_with_schema.na

df_csv_with_schema.show(5)

In [0]:
df_csv_with_schema.printSchema()

In [0]:
# Rename columns and add columns
from pyspark.sql.functions import lit, current_timestamp, col
Renamed_extra_df=df_csv_with_schema.withColumnRenamed("circuitId", "circuit_id") \
.withColumn("ingestionTime", current_timestamp()) \
.withColumn("env", lit("Production")) \
 .select(col("circuit_id"),col("ingestionTime"),col("env"))   
Renamed_extra_df.show(5)

In [0]:
# write to parquet file and read from parquet file
Renamed_extra_df.write.mode("overwrite").parquet("abfss://raw@databdls.dfs.core.windows.net/circuits_processed")
spark.read.parquet("abfss://processed@databdls.dfs.core.windows.net/circuits_processed").show()

# by default save as delta table even if it is parquet as we cant save parquet as managed table
Renamed_extra_df.write.mode("overwrite").saveAsTable("my_managed_parquet_table") 
display(spark.read.table("my_managed_parquet_table"))


In [0]:
# Write delta file 
Renamed_extra_df.write.mode("overwrite").format("delta").save("abfss://processed@databdls.dfs.core.windows.net/circuits_processed_delta")
display(spark.read.format("delta").load("abfss://processed@databdls.dfs.core.windows.net/circuits_processed_delta"))

In [0]:
# spark.sql("USE CATALOG databdev_silver_formula1_catalog")
spark.sql("USE CATALOG dev_silver_formula1_catalog")
spark.sql("USE SCHEMA processed_formula1_schema")
Renamed_extra_df.write.mode("overwrite").format("delta").saveAsTable("circuits_managed")
display(spark.read.table("circuits_managed"))